In [1]:
# This branch of qiskit is required:  pip install git+https://github.com/LNoorl/qiskit-terra.git@feature/sk-pass

import numpy as np
import dreamcoder as dc
from dreamcoder.domains.quantum_circuits.primitives import *
from dreamcoder.domains.quantum_circuits.tasks import *

import time
from tqdm import trange
import random
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [5]:
try:
    import binutil  # required to import from dreamcoder modules
except ModuleNotFoundError:
    import bin.binutil  # alt import if called as module

from dreamcoder.domains.quantum_algorithms.main import main
from dreamcoder.dreamcoder import commandlineArguments
from dreamcoder.utilities import numberOfCPUs

arguments = commandlineArguments(
    featureExtractor=None, # it was TowerCNN
    CPUs=numberOfCPUs(),
    helmholtzRatio=0.5,
    recognitionTimeout=6,
    iterations=6,
    a=3,
    structurePenalty=1,
    pseudoCounts=10,
    topK=2,
    maximumFrontier=5,
    extras=None,
    solver="bottom", 
    useRecognitionModel=False,
    enumerationTimeout=6,#-g
    compressor="pypy")   #ocaml, python, pypy  


CUDA is available?: True
using cuda?: True


usage: ipykernel_launcher.py [-h] [--resume RESUME] [-i ITERATIONS]
                             [-t ENUMERATIONTIMEOUT] [-R RECOGNITIONTIMEOUT]
                             [-RS RECOGNITIONSTEPS] [-k TOPK]
                             [-p PSEUDOCOUNTS] [-b AIC] [-l STRUCTUREPENALTY]
                             [-a ARITY] [-c CPUS] [--no-cuda]
                             [-m MAXIMUMFRONTIER] [--reuseRecognition]
                             [--recognition] [--ensembleSize ENSEMBLESIZE]
                             [-g] [-d] [--no-consolidation]
                             [--testingTimeout TESTINGTIMEOUT]
                             [--testEvery TESTEVERY] [--seed SEED]
                             [--activation {relu,sigmoid,tanh}]
                             [--solver {ocaml,pypy,bottom,bottom_unsound,bottom_simple,python}]
                             [-r HELMHOLTZRATIO]
                             [--compressor {pypy,rust,vs,pypy_vs,ocaml,memorize}]
                          

In [8]:
main(arguments)


Running EC on zpg003 @ 2022-05-26 19:22:46.566683 with 32 CPUs and parameters:
	 noConsolidation  =  False
	 iterations  =  6
	 enumerationTimeout  =  6
	 useRecognitionModel  =  False
	 topk_use_only_likelihood  =  False
	 pseudoCounts  =  10
	 aic  =  1.0
	 structurePenalty  =  1
	 arity  =  3
	 taskReranker  =  default
	 storeTaskMetrics  =  True
	 rewriteTaskMetrics  =  False
	 maximumFrontier  =  5
	 solver  =  bottom
	 topK  =  2
	 evaluationTimeout  =  0.01
	 cuda  =  True

Currently using this much memory: 394440704
Currently using this much memory: 394440704
Using a waking task batch of size: 19
pcfg compilation: distinct non terminals 2 ; distinct environments 2
start symbol: (tcircuit, (tcircuit,))

(tcircuit, (tcircuit,)) ::= mv	0x(tcircuit, (tcircuit,))		-2.0794415416798357
(tcircuit, (tcircuit,)) ::= mv_r	0x(tcircuit, (tcircuit,))		-2.0794415416798357
(tcircuit, (tcircuit,)) ::= minv	0x(tcircuit, (tcircuit,))		-2.0794415416798357
(tcircuit, (tcircuit,)) ::= h	0x(tcircuit,

AssertionError: 

# Some fixed-size circuits

## 1-qubit gates

X

In [4]:
# X
with QiskitTester(1) as QT:
    QT.circuit.x(QT.q(0))
QT.circuit.draw(), QT.transpile().draw()

x []


(   ┌───┐
 q: ┤ X ├
    └───┘,
 global phase: π
    ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐
 q: ┤ H ├┤ T ├┤ T ├┤ T ├┤ T ├┤ H ├
    └───┘└───┘└───┘└───┘└───┘└───┘)

Y

In [5]:
# Y
with QiskitTester(1) as QT:
    QT.circuit.y(QT.q(0))
QT.circuit.draw(), QT.transpile().draw()

y []


(   ┌───┐
 q: ┤ Y ├
    └───┘,
 global phase: 3π/4
    ┌───┐┌───┐┌───┐┌───┐┌─────┐┌─────┐┌───┐┌───┐┌───┐
 q: ┤ H ├┤ T ├┤ T ├┤ H ├┤ Tdg ├┤ Tdg ├┤ H ├┤ T ├┤ T ├
    └───┘└───┘└───┘└───┘└─────┘└─────┘└───┘└───┘└───┘)

Z

In [6]:
with QiskitTester(1) as QT:
    QT.circuit.z(QT.q(0))
QT.circuit.draw(), QT.transpile().draw()

z []


(   ┌───┐
 q: ┤ Z ├
    └───┘,
    ┌───┐┌───┐┌───┐┌───┐
 q: ┤ T ├┤ T ├┤ T ├┤ T ├
    └───┘└───┘└───┘└───┘)

T

In [7]:
# T = p(π/4)
with QiskitTester(1) as QT:
    QT.circuit.t(QT.q(0))
QT.circuit.draw(), QT.transpile().draw()

t []


(   ┌───┐
 q: ┤ T ├
    └───┘,
    ┌───┐
 q: ┤ T ├
    └───┘)

S

In [8]:
# S = p(π/2)
with QiskitTester(1) as QT:
    QT.circuit.s(0)
QT.circuit.draw(), QT.transpile().draw()


s []


(   ┌───┐
 q: ┤ S ├
    └───┘,
    ┌───┐┌───┐
 q: ┤ T ├┤ T ├
    └───┘└───┘)

$\sqrt X$

In [9]:
with QiskitTester(1) as QT:
    QT.circuit.sx(QT.q(0))
QT.circuit.draw(), QT.transpile().draw()

sx []


(   ┌────┐
 q: ┤ √X ├
    └────┘,
 global phase: π
    ┌───┐┌───┐┌───┐┌───┐
 q: ┤ H ├┤ T ├┤ T ├┤ H ├
    └───┘└───┘└───┘└───┘)

$\sqrt X ^\dag$

In [17]:
with QiskitTester(1) as QT:
    QT.circuit.sxdg(QT.q(0))
QT.circuit.draw(), QT.transpile().draw()

sxdg []


(   ┌──────┐
 q: ┤ √Xdg ├
    └──────┘,
 global phase: π
    ┌───┐┌─────┐┌─────┐┌───┐
 q: ┤ H ├┤ Tdg ├┤ Tdg ├┤ H ├
    └───┘└─────┘└─────┘└───┘)

Consistency checks

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.append(qk.circuit.library.PhaseGate(np.pi/2),(0,))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(), circuit2.draw(), discretized.draw()

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.append(qk.circuit.library.PhaseGate(3*np.pi/4),(0,))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(), circuit2.draw(), discretized.draw()

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.tdg(0)
    QT.circuit.tdg(0)
QT.circuit=pm.run(QT.circuit)
transpiled=qk.transpile(QT.circuit, backend, basis_gates)
discretized = skd(transpiled)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.append(qk.circuit.library.PhaseGate(3*np.pi/2),(0,))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(), circuit2.draw(), discretized.draw()

In [ ]:
with QiskitTester(1) as QT:
    QT.circuit.append(qk.circuit.library.PhaseGate(5*np.pi/2),(0,))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(), circuit2.draw(), discretized.draw()

In [ ]:
# (just as a test)
# we should be able to transpile this one
with QiskitTester(1) as QT:
    QT.circuit.p(np.pi/8,0)
    QT.circuit.p(np.pi/8,0)
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [37]:
# we should be able to transpile this one
with QiskitTester(1) as QT:
    QT.circuit.p(np.pi/8,0)
    QT.circuit.p(np.pi/8,0)
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

u1 [0.7853981633974483]


(   ┌────────┐┌────────┐
 q: ┤ P(π/8) ├┤ P(π/8) ├
    └────────┘└────────┘,
    ┌─────────┐
 q: ┤ U1(π/4) ├
    └─────────┘,
    ┌───┐
 q: ┤ T ├
    └───┘)

## 2-qubit gates

Controlled-Y

In [18]:
with QiskitTester(2) as QT:
    QT.circuit.cy(QT.q(0),QT.q(1))
QT.circuit.draw(), QT.transpile().draw()

cy []


(     ┌───┐
 q_0: ┤ Y ├
      └─┬─┘
 q_1: ──■──
           ,
      ┌─────┐┌─────┐┌───┐┌───┐┌───┐
 q_0: ┤ Tdg ├┤ Tdg ├┤ X ├┤ T ├┤ T ├
      └─────┘└─────┘└─┬─┘└───┘└───┘
 q_1: ────────────────■────────────
                                   )

CZ

In [19]:
with QiskitTester(2) as QT:
    QT.circuit.cz(QT.q(0),QT.q(1))
QT.circuit.draw(), QT.transpile().draw()

cz []


(        
 q_0: ─■─
       │ 
 q_1: ─■─
         ,
      ┌───┐┌───┐┌───┐
 q_0: ┤ H ├┤ X ├┤ H ├
      └───┘└─┬─┘└───┘
 q_1: ───────■───────
                     )

SWAP

In [20]:
with QiskitTester(2) as QT:
    QT.circuit.swap(QT.q(0),QT.q(1))
QT.circuit.draw(), QT.transpile().draw()

swap []


(        
 q_0: ─X─
       │ 
 q_1: ─X─
         ,
      ┌───┐     ┌───┐
 q_0: ┤ X ├──■──┤ X ├
      └─┬─┘┌─┴─┐└─┬─┘
 q_1: ──■──┤ X ├──■──
           └───┘     )

iSWAP

In [21]:
with QiskitTester(2) as QT:
    QT.circuit.iswap(QT.q(0),QT.q(1))
QT.circuit.draw(), QT.transpile().draw()

s []
u2 [0.0, -1.5707963267948966]
cx []
cx []
h []


(     ┌────────┐
 q_0: ┤1       ├
      │  Iswap │
 q_1: ┤0       ├
      └────────┘,
 global phase: π
      ┌───┐┌───┐     ┌───┐     ┌───┐
 q_0: ┤ T ├┤ T ├─────┤ X ├──■──┤ H ├
      ├───┤├───┤┌───┐└─┬─┘┌─┴─┐└───┘
 q_1: ┤ T ├┤ T ├┤ H ├──■──┤ X ├─────
      └───┘└───┘└───┘     └───┘     )

Controlled S

In [22]:
with QiskitTester(2) as QT:
    QT.circuit.append(qk.circuit.library.SGate().control(1), (0,1))
QT.circuit.draw(), QT.transpile().draw()

cp [1.5707963267948966]


(          
 q_0: ──■──
      ┌─┴─┐
 q_1: ┤ S ├
      └───┘,
      ┌───┐                      
 q_0: ┤ T ├──■───────────■───────
      └───┘┌─┴─┐┌─────┐┌─┴─┐┌───┐
 q_1: ─────┤ X ├┤ Tdg ├┤ X ├┤ T ├
           └───┘└─────┘└───┘└───┘)

DCX
(Double CNOT)

In [23]:
with QiskitTester(2) as QT:
    QT.circuit.dcx(QT.q(0),QT.q(1))
QT.circuit.draw(), QT.transpile().draw()

cx []
cx []


(     ┌──────┐
 q_0: ┤1     ├
      │  Dcx │
 q_1: ┤0     ├
      └──────┘,
      ┌───┐     
 q_0: ┤ X ├──■──
      └─┬─┘┌─┴─┐
 q_1: ──■──┤ X ├
           └───┘)

CH
(Controlled Hadamard)

In [24]:
with QiskitTester(2) as QT:
    QT.circuit.ch(QT.q(0),QT.q(1))
QT.circuit.draw(), QT.transpile().draw()

u2 [0.7853981633974483, -1.5707963267948966]
cx []
u2 [-1.5707963267948966, 2.356194490192345]


(     ┌───┐
 q_0: ┤ H ├
      └─┬─┘
 q_1: ──■──
           ,
 global phase: π
      ┌───┐┌───┐┌───┐┌───┐┌───┐┌─────┐┌───┐┌─────┐┌─────┐
 q_0: ┤ T ├┤ T ├┤ H ├┤ T ├┤ X ├┤ Tdg ├┤ H ├┤ Tdg ├┤ Tdg ├
      └───┘└───┘└───┘└───┘└─┬─┘└─────┘└───┘└─────┘└─────┘
 q_1: ──────────────────────■────────────────────────────
                                                         )

### Consistency checks

In [ ]:
# Controlled Phase(α) requires to be able to do Phase(α/2)
with QiskitTester(2) as QT:
    QT.circuit.append(qk.circuit.library.CPhaseGate(3*np.pi/2),(0,1))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(), circuit2.draw(), discretized.draw()

In [ ]:
# This is CS, but can we transpile it?
with QiskitTester(2) as QT:
    QT.circuit.cp(np.pi/2,QT.q(0),QT.q(1))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

## 3-qubit gates

Controlled-Controlled X

In [25]:
with QiskitTester(3) as QT:
    QT.circuit.ccx(QT.q(0),QT.q(1),QT.q(2))
QT.circuit.draw(), QT.transpile().draw()

ccx []


(     ┌───┐
 q_0: ┤ X ├
      └─┬─┘
 q_1: ──■──
        │  
 q_2: ──■──
           ,
      ┌───┐┌───┐┌─────┐┌───┐┌───┐┌───┐┌─────┐┌───┐┌───┐ ┌───┐      
 q_0: ┤ H ├┤ X ├┤ Tdg ├┤ X ├┤ T ├┤ X ├┤ Tdg ├┤ X ├┤ T ├─┤ H ├──────
      └───┘└─┬─┘└─────┘└─┬─┘└───┘└─┬─┘└┬───┬┘└─┬─┘├───┤┌┴───┴┐┌───┐
 q_1: ───────■───────────┼─────────■───┤ T ├───┼──┤ X ├┤ Tdg ├┤ X ├
                         │             └───┘   │  └─┬─┘└┬───┬┘└─┬─┘
 q_2: ───────────────────■─────────────────────■────■───┤ T ├───■──
                                                        └───┘      )

Controlled-Swap

In [26]:
with QiskitTester(3) as QT:
    QT.circuit.cswap(QT.q(0),QT.q(1),QT.q(2))
QT.circuit.draw(), QT.transpile().draw()

cswap []


(        
 q_0: ─X─
       │ 
 q_1: ─X─
       │ 
 q_2: ─■─
         ,
           ┌───┐┌───┐┌─────┐┌───┐┌───┐┌───┐┌─────┐┌───┐┌───┐ ┌───┐           
 q_0: ──■──┤ H ├┤ X ├┤ Tdg ├┤ X ├┤ T ├┤ X ├┤ Tdg ├┤ X ├┤ T ├─┤ H ├────────■──
      ┌─┴─┐└───┘└─┬─┘└─────┘└─┬─┘└───┘└─┬─┘└┬───┬┘└─┬─┘├───┤┌┴───┴┐┌───┐┌─┴─┐
 q_1: ┤ X ├───────■───────────┼─────────■───┤ T ├───┼──┤ X ├┤ Tdg ├┤ X ├┤ X ├
      └───┘                   │             └───┘   │  └─┬─┘└┬───┬┘└─┬─┘└───┘
 q_2: ────────────────────────■─────────────────────■────■───┤ T ├───■───────
                                                             └───┘           )

## Variable size / parametric

Permutation

In [27]:
with QiskitTester(3) as QT:
    QT.circuit.append(qk.circuit.library.Permutation(3,[2,0,1]), 
                      (QT.q(0),QT.q(1),QT.q(2)))
QT.circuit.draw(), QT.transpile().draw()

swap []
swap []


(     ┌──────────────────────┐
 q_0: ┤2                     ├
      │                      │
 q_1: ┤1 permutation_[2,0,1] ├
      │                      │
 q_2: ┤0                     ├
      └──────────────────────┘,
      ┌───┐     ┌───┐               
 q_0: ┤ X ├──■──┤ X ├───────────────
      └─┬─┘┌─┴─┐└─┬─┘┌───┐     ┌───┐
 q_1: ──■──┤ X ├──■──┤ X ├──■──┤ X ├
           └───┘     └─┬─┘┌─┴─┐└─┬─┘
 q_2: ─────────────────■──┤ X ├──■──
                          └───┘     )

InnerProduct
(but only specific sizes)

In [ ]:
with QiskitTester(4) as QT:
    QT.circuit.append(qk.circuit.library.InnerProduct(2), 
                      (QT.q(0),QT.q(1),QT.q(2),QT.q(3)))
QT.circuit.draw(), QT.transpile().draw()

cz []
cz []


(     ┌────────────────┐
 q_0: ┤3               ├
      │                │
 q_1: ┤2               ├
      │  inner_product │
 q_2: ┤1               ├
      │                │
 q_3: ┤0               ├
      └────────────────┘,
      ┌───┐┌───┐┌───┐     
 q_0: ┤ H ├┤ X ├┤ H ├─────
      ├───┤└─┬─┘├───┤┌───┐
 q_1: ┤ H ├──┼──┤ X ├┤ H ├
      └───┘  │  └─┬─┘└───┘
 q_2: ───────■────┼───────
                  │       
 q_3: ────────────■───────
                          )

AND

In [28]:
with QiskitTester(3) as QT:
    QT.circuit.append(qk.circuit.library.AND(2), 
                      (QT.q(0),QT.q(1),QT.q(2)))
QT.circuit.draw(), QT.transpile().draw()

ccx []


(     ┌──────┐
 q_0: ┤2     ├
      │      │
 q_1: ┤1 and ├
      │      │
 q_2: ┤0     ├
      └──────┘,
      ┌───┐┌───┐┌─────┐┌───┐┌───┐┌───┐┌─────┐┌───┐┌───┐ ┌───┐      
 q_0: ┤ H ├┤ X ├┤ Tdg ├┤ X ├┤ T ├┤ X ├┤ Tdg ├┤ X ├┤ T ├─┤ H ├──────
      └───┘└─┬─┘└─────┘└─┬─┘└───┘└─┬─┘└┬───┬┘└─┬─┘├───┤┌┴───┴┐┌───┐
 q_1: ───────■───────────┼─────────■───┤ T ├───┼──┤ X ├┤ Tdg ├┤ X ├
                         │             └───┘   │  └─┬─┘└┬───┬┘└─┬─┘
 q_2: ───────────────────■─────────────────────■────■───┤ T ├───■──
                                                        └───┘      )

OR

In [29]:
with QiskitTester(3) as QT:
    QT.circuit.append(qk.circuit.library.OR(2), 
                      (QT.q(0),QT.q(1),QT.q(2)))
QT.circuit.draw(), QT.transpile().draw()

x []
x []
x []
ccx []
x []
x []


(     ┌─────┐
 q_0: ┤2    ├
      │     │
 q_1: ┤1 or ├
      │     │
 q_2: ┤0    ├
      └─────┘,
 global phase: π
      ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌─────┐┌───┐┌───┐┌───┐┌─────┐┌───┐»
 q_0: ┤ H ├┤ T ├┤ T ├┤ T ├┤ T ├┤ H ├┤ H ├┤ X ├┤ Tdg ├┤ X ├┤ T ├┤ X ├┤ Tdg ├┤ X ├»
      ├───┤├───┤├───┤├───┤├───┤├───┤└───┘└─┬─┘└─────┘└─┬─┘└───┘└─┬─┘└┬───┬┘└─┬─┘»
 q_1: ┤ H ├┤ T ├┤ T ├┤ T ├┤ T ├┤ H ├───────■───────────┼─────────■───┤ T ├───┼──»
      ├───┤├───┤├───┤├───┤├───┤├───┤                   │             └───┘   │  »
 q_2: ┤ H ├┤ T ├┤ T ├┤ T ├┤ T ├┤ H ├───────────────────■─────────────────────■──»
      └───┘└───┘└───┘└───┘└───┘└───┘                                            »
 «     ┌───┐ ┌───┐                                    
 «q_0: ┤ T ├─┤ H ├────────────────────────────────────
 «     ├───┤┌┴───┴┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐
 «q_1: ┤ X ├┤ Tdg ├┤ X ├┤ H ├┤ T ├┤ T ├┤ T ├┤ T ├┤ H ├
 «     └─┬─┘└┬───┬┘└─┬─┘├───┤├───┤├───┤├───┤├───┤├───┤
 «q_2: ──■───┤ T ├───■──┤ H ├┤ T ├┤

CarryAdder (CDKM)

In [40]:
with QiskitTester(6) as QT:
    QT.circuit.append(qk.circuit.library.CDKMRippleCarryAdder(2), 
                      (QT.q(0),QT.q(1),QT.q(2),QT.q(3),QT.q(4),QT.q(5)))
QT.circuit.draw(), QT.transpile().draw()

cx []
cx []
cx []
ccx []
cx []
ccx []
cx []
ccx []
cx []
cx []
ccx []
cx []
cx []


(     ┌───────────────────────┐
 q_0: ┤5                      ├
      │                       │
 q_1: ┤4                      ├
      │                       │
 q_2: ┤3                      ├
      │  CDKMRippleCarryAdder │
 q_3: ┤2                      ├
      │                       │
 q_4: ┤1                      ├
      │                       │
 q_5: ┤0                      ├
      └───────────────────────┘,
                                                                           »
 q_0: ─────────────────────────────────────────────────────────────────────»
      ┌───┐                                                                »
 q_1: ┤ X ├────────────────────────────────────────────────────────────────»
      └─┬─┘┌───┐                                      ┌───┐           ┌───┐»
 q_2: ──┼──┤ X ├────────────■─────────────────────■───┤ T ├───────────┤ X ├»
        │  └─┬─┘            │                     │   └───┘           └─┬─┘»
 q_3: ──■────┼──────────────┼───────────────

CarryAdder (VBER)

In [41]:
with QiskitTester(7) as QT:
    QT.circuit.append(qk.circuit.library.VBERippleCarryAdder(2), 
                      (QT.q(0),QT.q(1),QT.q(2),QT.q(3),QT.q(4),QT.q(5),QT.q(6)))
QT.circuit.draw(), QT.transpile().draw()

ccx []
cx []
ccx []
cx []
ccx []
ccx []
cx []
ccx []
cx []
ccx []
cx []
cx []


(     ┌──────────────────────┐
 q_0: ┤6                     ├
      │                      │
 q_1: ┤5                     ├
      │                      │
 q_2: ┤4                     ├
      │                      │
 q_3: ┤3 VBERippleCarryAdder ├
      │                      │
 q_4: ┤2                     ├
      │                      │
 q_5: ┤1                     ├
      │                      │
 q_6: ┤0                     ├
      └──────────────────────┘,
      ┌───┐     ┌───┐┌─────┐     ┌───┐┌───┐     ┌───┐┌─────┐     ┌───┐┌───┐»
 q_0: ┤ H ├─────┤ X ├┤ Tdg ├─────┤ X ├┤ T ├─────┤ X ├┤ Tdg ├─────┤ X ├┤ T ├»
      ├───┤┌───┐└─┬─┘├─────┤┌───┐└─┬─┘├───┤┌───┐└─┬─┘├─────┤┌───┐└─┬─┘├───┤»
 q_1: ┤ H ├┤ X ├──┼──┤ Tdg ├┤ X ├──┼──┤ T ├┤ X ├──┼──┤ Tdg ├┤ X ├──┼──┤ T ├»
      └───┘└─┬─┘  │  └─────┘└─┬─┘  │  └───┘└─┬─┘  │  └┬───┬┘└─┬─┘  │  ├───┤»
 q_2: ───────■────┼───────────┼────┼─────────■────┼───┤ T ├───┼────┼──┤ X ├»
                  │           │    │              │   ├───┤   │    │  └─

Weighted Adder
(long)

In [42]:
with QiskitTester(5) as QT:
    QT.circuit.append(qk.circuit.library.WeightedAdder(2), 
                      (QT.q(0),QT.q(1),QT.q(2),QT.q(3),QT.q(4)))
QT.circuit.draw(), QT.transpile().draw()

ccx []
cx []
x []
ccx []
ccx []
x []
ccx []
cx []
x []
ccx []
ccx []
x []


(     ┌────────┐
 q_0: ┤4       ├
      │        │
 q_1: ┤3       ├
      │        │
 q_2: ┤2 adder ├
      │        │
 q_3: ┤1       ├
      │        │
 q_4: ┤0       ├
      └────────┘,
      ┌───┐┌───┐┌─────┐┌───┐┌───┐┌───┐┌─────┐┌───┐┌───┐ ┌───┐                  »
 q_0: ┤ H ├┤ X ├┤ Tdg ├┤ X ├┤ T ├┤ X ├┤ Tdg ├┤ X ├┤ T ├─┤ H ├───■──────────────»
      ├───┤└─┬─┘└─────┘└─┬─┘└───┘└─┬─┘└─────┘└─┬─┘└───┘ └───┘ ┌─┴─┐┌─────┐     »
 q_1: ┤ H ├──┼───────────┼─────────┼───────────┼──────────────┤ X ├┤ Tdg ├─────»
      └───┘  │           │         │   ┌───┐   │  ┌───┐┌─────┐├───┤└┬───┬┘┌───┐»
 q_2: ───────■───────────┼─────────■───┤ T ├───┼──┤ X ├┤ Tdg ├┤ H ├─┤ T ├─┤ T ├»
                         │             └───┘   │  └─┬─┘└─────┘└───┘ └───┘ └───┘»
 q_3: ───────────────────┼─────────────────────┼────┼──────────────────────────»
                         │                     │    │   ┌───┐                  »
 q_4: ───────────────────■─────────────────────■────■───┤ T ├──────────────────»
  

In [45]:
with QiskitTester(4) as QT:
    QT.circuit.append(qk.circuit.library.IntegerComparator(2, value=3), 
                      (QT.q(0),QT.q(1),QT.q(2),QT.q(3),))
QT.circuit.draw(), QT.transpile().draw()

cx []
ccx []
cx []


(     ┌──────┐
 q_0: ┤3     ├
      │      │
 q_1: ┤2     ├
      │  cmp │
 q_2: ┤1     ├
      │      │
 q_3: ┤0     ├
      └──────┘,
           ┌───┐                            ┌───┐           ┌───┐┌─────┐┌───┐»
 q_0: ─────┤ X ├──■─────────────────────■───┤ T ├───────────┤ X ├┤ Tdg ├┤ X ├»
      ┌───┐└─┬─┘┌─┴─┐┌─────┐┌───┐┌───┐┌─┴─┐┌┴───┴┐┌───┐┌───┐└─┬─┘└┬───┬┘└─┬─┘»
 q_1: ┤ H ├──┼──┤ X ├┤ Tdg ├┤ X ├┤ T ├┤ X ├┤ Tdg ├┤ X ├┤ T ├──┼───┤ H ├───┼──»
      └───┘  │  └───┘└─────┘└─┬─┘└───┘└───┘└─────┘└─┬─┘└───┘  │   ├───┤   │  »
 q_2: ───────┼────────────────■─────────────────────■─────────■───┤ T ├───■──»
             │                                                    └───┘      »
 q_3: ───────■───────────────────────────────────────────────────────────────»
                                                                             »
 «     ┌───┐
 «q_0: ┤ X ├
 «     └─┬─┘
 «q_1: ──┼──
 «       │  
 «q_2: ──┼──
 «       │  
 «q_3: ──■──
 «          )

Reciprocal

In [47]:
with QiskitTester(3) as QT:
    QT.circuit.append(qk.circuit.library.ExactReciprocal(2, scaling=1.5), 
                      (QT.q(0),QT.q(1),QT.q(2)))
QT.circuit.draw(), QT.transpile().draw()

cx []
cx []
cx []
cx []


(     ┌──────┐
 q_0: ┤2     ├
      │      │
 q_1: ┤1 1/x ├
      │      │
 q_2: ┤0     ├
      └──────┘,
      ┌───┐┌───┐┌───┐┌───┐
 q_0: ┤ X ├┤ X ├┤ X ├┤ X ├
      └─┬─┘└─┬─┘└─┬─┘└─┬─┘
 q_1: ──┼────■────┼────■──
        │         │       
 q_2: ──■─────────■───────
                          )

'Hidden Linear Function'

In [61]:
with QiskitTester(3) as QT:
    QT.circuit.append(qk.circuit.library.HiddenLinearFunction([[1, 1, 0], 
                                                               [1, 0, 1], 
                                                               [0, 1, 1]]), 
                      (QT.q(0),QT.q(1),QT.q(2)))
QT.circuit.draw(), QT.transpile().draw()

h []
h []
h []
cz []
cz []
u2 [0.0, -1.5707963267948966]
h []
u2 [0.0, -1.5707963267948966]


(     ┌───────────────────────────────────┐
 q_0: ┤2                                  ├
      │                                   │
 q_1: ┤1 hlf: [[1 1 0]
  [1 0 1]
  [0 1 1]] ├
      │                                   │
 q_2: ┤0                                  ├
      └───────────────────────────────────┘,
      ┌───┐┌───┐          ┌───┐┌───┐┌───┐┌───┐┌───┐
 q_0: ┤ H ├┤ H ├──────────┤ X ├┤ H ├┤ T ├┤ T ├┤ H ├
      ├───┤├───┤┌───┐┌───┐└─┬─┘├───┤└───┘└───┘└───┘
 q_1: ┤ H ├┤ H ├┤ X ├┤ H ├──■──┤ H ├───────────────
      ├───┤└───┘└─┬─┘├───┤┌───┐├───┤               
 q_2: ┤ H ├───────■──┤ T ├┤ T ├┤ H ├───────────────
      └───┘          └───┘└───┘└───┘               )

Grover operator
(depends on the oracle)

In [71]:
# Grover Operator 
with QiskitTester(2) as QT:
    oracle = QuantumCircuit(2)
    oracle.z(0)
    
    QT.circuit.append(qk.circuit.library.GroverOperator(oracle), 
                      (QT.q(0),QT.q(1)))
QT.circuit.draw(), QT.transpile().draw()

u1 [-3.141592653589793]
u2 [0.0, -3.141592653589793]
cx []
u1 [-3.141592653589793]
u2 [-3.141592653589793, -3.141592653589793]


(     ┌────┐
 q_0: ┤1   ├
      │  Q │
 q_1: ┤0   ├
      └────┘,
 global phase: π
      ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐     
 q_0: ┤ T ├┤ T ├┤ T ├┤ T ├┤ X ├┤ T ├┤ T ├┤ T ├┤ T ├─────
      ├───┤└───┘└───┘└───┘└─┬─┘├───┤├───┤├───┤├───┤┌───┐
 q_1: ┤ H ├─────────────────■──┤ H ├┤ T ├┤ T ├┤ T ├┤ T ├
      └───┘                    └───┘└───┘└───┘└───┘└───┘)

## Quantum State Preparation

In [ ]:
# In general: 
# GHZ and W state preparation algorithms https://arxiv.org/pdf/1807.05572.pdf
# W states: https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/awards/teach_me_qiskit_2018/w_state/W%20State%201%20-%20Multi-Qubit%20Systems.ipynb
# Cluster states https://www.cs.umd.edu/class/spring2018/cmsc457/report/Cluster_State_Quantum_Computing.pdf

Bell
(2-qubit)

In [50]:
# Bell 
with QiskitTester(2) as QT:
    QT.circuit.h(QT.q(0))
    QT.circuit.cx(QT.q(0),QT.q(1))
QT.circuit.draw(),transpiled.draw(),discretized.draw()

(          ┌───┐
 q_0: ─────┤ X ├
      ┌───┐└─┬─┘
 q_1: ┤ H ├──■──
      └───┘     ,
                          
 q_0: ────────────────────
                          
 q_1: ────────────────────
      ┌───┐┌───┐┌───┐┌───┐
 q_2: ┤ X ├┤ X ├┤ X ├┤ X ├
      └─┬─┘└─┬─┘└─┬─┘└─┬─┘
 q_3: ──┼────■────┼────■──
        │         │       
 q_4: ──■─────────■───────
                          ,
                          
 q_0: ────────────────────
                          
 q_1: ────────────────────
      ┌───┐┌───┐┌───┐┌───┐
 q_2: ┤ X ├┤ X ├┤ X ├┤ X ├
      └─┬─┘└─┬─┘└─┬─┘└─┬─┘
 q_3: ──┼────■────┼────■──
        │         │       
 q_4: ──■─────────■───────
                          )

GHZ
(3-qubit, but can be generalized)

In [52]:
# GHZ
with QiskitTester(3) as QT:
    QT.circuit.h(QT.q(0))
    QT.circuit.cx(QT.q(0),QT.q(1))
    QT.circuit.cx(QT.q(0),QT.q(2))
QT.circuit.draw(), QT.transpile().draw()

h []
cx []
cx []


(               ┌───┐
 q_0: ──────────┤ X ├
           ┌───┐└─┬─┘
 q_1: ─────┤ X ├──┼──
      ┌───┐└─┬─┘  │  
 q_2: ┤ H ├──■────■──
      └───┘          ,
                ┌───┐
 q_0: ──────────┤ X ├
           ┌───┐└─┬─┘
 q_1: ─────┤ X ├──┼──
      ┌───┐└─┬─┘  │  
 q_2: ┤ H ├──■────■──
      └───┘          )

In [ ]:
# W this is too long
with QiskitTester(3) as QT:
    QT.circuit.ry(2*np.arccos(1/np.sqrt(3)),QT.q(0))
    QT.circuit.h(QT.q(1))
    QT.circuit.cnot(QT.q(1),QT.q(2))
    QT.circuit.cnot(QT.q(0),QT.q(1))
    QT.circuit.x(QT.q(0))
QT.circuit.draw(), QT.transpile().draw()

In [59]:
# Graph state
with QiskitTester(5) as QT:
    QT.circuit.append(qk.circuit.library.GraphState([[0., 1., 0., 0., 1.,],
                                                     [1., 0., 1., 0., 0.,],
                                                     [0., 1., 0., 1., 0.],
                                                     [0., 0., 1., 0., 1.],
                                                     [1., 0., 0., 1., 0.]]), 
                      (QT.q(0),QT.q(1),QT.q(2),QT.q(3),QT.q(4)))
QT.circuit.draw(), QT.transpile().draw()

h []
h []
h []
h []
h []
cz []
cz []
cz []
cz []
cz []


(     »
 q_0: »
      »
 q_1: »
      »
 q_2: »
      »
 q_3: »
      »
 q_4: »
      »
 «     ┌────────────────────────────────────────────────────────────────────────────────────────────────────┐
 «q_0: ┤4                                                                                                   ├
 «     │                                                                                                    │
 «q_1: ┤3                                                                                                   ├
 «     │                                                                                                    │
 «q_2: ┤2 graph: [[0. 1. 0. 0. 1.]
  [1. 0. 1. 0. 0.]
  [0. 1. 0. 1. 0.]
  [0. 0. 1. 0. 1.]
  [1. 0. 0. 1. 0.]] ├
 «     │                                                                                                    │
 «q_3: ┤1                                                                                                   ├
 «     │                    

# Too Long

In [2]:
## WE NEED CP(pi/8) FOR QFT!
# It works but it is huge!
with QiskitTester(3) as QT:
    QT.circuit.append(qk.circuit.library.QFT(3), 
                      (QT.q(0),QT.q(1),QT.q(2)))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

h []
cp [1.5707963267948966]
h []
cp [0.7853981633974483]
cp [1.5707963267948966]
h []
swap []


(     ┌──────┐
 q_0: ┤2     ├
      │      │
 q_1: ┤1 QFT ├
      │      │
 q_2: ┤0     ├
      └──────┘,
      ┌───┐                                        
 q_0: ┤ H ├─■─────────────■──────────────────────X─
      └───┘ │P(π/2) ┌───┐ │                      │ 
 q_1: ──────■───────┤ H ├─┼────────■─────────────┼─
                    └───┘ │P(π/4)  │P(π/2) ┌───┐ │ 
 q_2: ────────────────────■────────■───────┤ H ├─X─
                                           └───┘   ,
 global phase: 3.7306
      ┌───┐┌───┐                 ┌───┐┌─────┐┌───┐┌─────┐┌───┐┌─────┐┌───┐»
 q_0: ┤ H ├┤ T ├──■───────────■──┤ H ├┤ Tdg ├┤ H ├┤ Tdg ├┤ H ├┤ Tdg ├┤ H ├»
      └───┘└───┘┌─┴─┐┌─────┐┌─┴─┐├───┤└┬───┬┘├───┤└─────┘└───┘└─────┘└───┘»
 q_1: ──────────┤ X ├┤ Tdg ├┤ X ├┤ T ├─┤ H ├─┤ T ├────────────────────────»
                └───┘└─────┘└───┘└───┘ └───┘ └───┘                        »
 q_2: ────────────────────────────────────────────────────────────────────»
                                                   

In [ ]:
## WE NEED CP(pi/8) FOR QFT!
# It works but it is huge!
with QiskitTester(4) as QT:
    QT.circuit.append(qk.circuit.library.QFT(4), 
                      (QT.q(0),QT.q(1),QT.q(2),QT.q(3)))
QT.circuit.draw(), QT.transpile().draw()

In [ ]:
# very long!
with QiskitTester(5) as QT:
    QT.circuit.append(qk.circuit.library.QuadraticForm(2, linear=[1,-2], 
                                                       quadratic=[[1,2],[2,0.5]]), 
                      (QT.q(0),QT.q(1),QT.q(2),QT.q(3),))
QT.circuit.draw(), QT.transpile().draw()

In [ ]:
# Too long
with QiskitTester(4) as QT:
    QT.circuit.append(qk.circuit.library.IQP([[6, 5, 3], [5, 4, 5], [3, 5, 1]]), 
                      (QT.q(0),QT.q(1),QT.q(2)))
QT.circuit.draw(), QT.transpile().draw()

In [ ]:
# Huge
# U(π/2,0,π,0)  not found
with QiskitTester(9) as QT:
    QT.circuit.append(qk.circuit.library.HRSCumulativeMultiplier(2), 
                      (QT.q(0),QT.q(1),QT.q(2),QT.q(3),QT.q(4),QT.q(5),QT.q(6),QT.q(7),QT.q(8)))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
#!Huge
with QiskitTester(8) as QT:
    QT.circuit.append(qk.circuit.library.RGQFTMultiplier(2), 
                      (QT.q(0),QT.q(1),QT.q(2),QT.q(3),QT.q(4),QT.q(5),QT.q(6),QT.q(7)))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
#incredibly huge
from qiskit import algorithms
circuit = qk.algorithms.Shor().construct_circuit(7)
transpiled=qk.transpile(circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
# Other algorithms - but should be evaluated with reasonable arguments!
# https://qiskit.org/documentation/apidoc/algorithms.html

In [ ]:
# Quantum Volume (seems to be unfeasible)

# Invalid

In [ ]:
with QiskitTester(2) as QT:
    QT.circuit.csx(QT.q(0),QT.q(1))
QT.result
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

csx []


(     ┌────┐
 q_0: ┤ Sx ├
      └─┬──┘
 q_1: ──■───
            ,
      ┌────┐
 q_0: ┤ Sx ├
      └─┬──┘
 q_1: ──■───
            ,
      ┌────┐
 q_0: ┤ Sx ├
      └─┬──┘
 q_1: ──■───
            )

In [ ]:
# Too long
with QiskitTester(1) as QT:
    QT.circuit.p(np.pi/8,QT.q(0))
QT.circuit.draw(), QT.transpile().draw()


In [ ]:
with QiskitTester(4) as QT:
    QT.circuit.append(qk.circuit.library.C3XGate(), (0,1,2,3))
QT.circuit.draw()
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(2) as QT:
    QT.circuit.ecr(QT.q(0),QT.q(1))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
# no idea of how this works
with QiskitTester(2) as QT:
    QT.circuit.append(qk.circuit.library.Diagonal([1,1]), 
                      (QT.q(0),))
transpiled=qk.transpile(QT.circuit, backend)
circuit2 = pm.run(transpiled)
discretized = skd(circuit2)
QT.circuit.draw(),transpiled.draw(),discretized.draw()

In [ ]:
with QiskitTester(3) as QT:
    QT.circuit.append(qk.circuit.library.FourierChecking([1,-1,1,1,1,-1,1,-1],[-1,-1,-1,-1,1,1,1,1]), 
                      (QT.q(0),QT.q(1),QT.q(2)))
QT.circuit.draw(), QT.transpile().draw()

In [ ]:
with QiskitTester(3) as QT:
    QT.circuit.append(qk.circuit.library.PhaseEstimation(2,qk.circuit.library.XGate()), 
                      (QT.q(0),QT.q(1),QT.q(2)))
QT.circuit.draw(), QT.transpile().draw()

- How to choose circuit size when enumerating
- Observational equivalence and circuit size